In [1]:
%matplotlib inline

import gensim
import logging
import math
import nltk
import os
import pyLDAvis.gensim_models
import random
import re
import spacy
import json
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import cm

from imageio import imread
from wordcloud import WordCloud

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
from nltk.corpus import stopwords # est-ce que c'est le même de sklearn ?
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
from nltk.tokenize import RegexpTokenizer

In [2]:
with open(os.path.join('posts_maior_propre.json'), 'r', encoding = 'UTF-8') as fin_M2 :
    data = json.load(fin_M2)

In [3]:
type(data)

list

In [4]:
len(data)

110895

In [5]:
data[1377]

"\n\n\n\nMarchant dans Londres, ville qu’elle connaît bien et qu’elle sillonne depuis de nombreuses années, Sylvie Doizelet a pris, pendant ses récents séjours, des séries de photos qui disent les dangers, interdictions, dérapages, risques de chute, de contamination et de mort qui guettent l’imprudent qui oublierait de prendre au sérieux tous ces rappels.\n\nIls fleurissent sur de multiples panneaux. On les trouve partout : des terrains vagues au cœur de la ville en passant par les trottoirs, les grilles, les cages d’escalier, les façades de verre, les quais, les esplanades, les souterrains… Façon pour les uns de se prémunir et pour les autres de trembloter en se méfiant de ces incessants périls susceptibles de transformer les éventuelles (et de plus en plus problématiques) flâneries en guet-apens.\n\nCes dangers inhérents à la capitale londonienne, Jean-Claude Pirotte a choisi de les contourner sans s’en laisser conter. Ces pancartes vont, au contraire, l’inciter à se promener en conv

In [6]:
index = ''
a = ''
list_clean_content = []

for j in range(len(data)) :
    index = data[j]
    a = index.replace( "\n", "")
    list_clean_content.append(a)

In [7]:
list_clean_content[1377]

"Marchant dans Londres, ville qu’elle connaît bien et qu’elle sillonne depuis de nombreuses années, Sylvie Doizelet a pris, pendant ses récents séjours, des séries de photos qui disent les dangers, interdictions, dérapages, risques de chute, de contamination et de mort qui guettent l’imprudent qui oublierait de prendre au sérieux tous ces rappels.Ils fleurissent sur de multiples panneaux. On les trouve partout : des terrains vagues au cœur de la ville en passant par les trottoirs, les grilles, les cages d’escalier, les façades de verre, les quais, les esplanades, les souterrains… Façon pour les uns de se prémunir et pour les autres de trembloter en se méfiant de ces incessants périls susceptibles de transformer les éventuelles (et de plus en plus problématiques) flâneries en guet-apens.Ces dangers inhérents à la capitale londonienne, Jean-Claude Pirotte a choisi de les contourner sans s’en laisser conter. Ces pancartes vont, au contraire, l’inciter à se promener en convoquant, dans les

In [8]:
from stop_words import get_stop_words
fr_stop_words = get_stop_words('french')
fr_stop_words

['a',
 'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'alors',
 'as',
 'au',
 'aucun',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aussi',
 'autre',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avant',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avoir',
 'avons',
 'ayant',
 'ayez',
 'ayons',
 'bon',
 'car',
 'ce',
 'ceci',
 'cela',
 'ces',
 'cet',
 'cette',
 'ceux',
 'chaque',
 'ci',
 'comme',
 'comment',
 'd',
 'dans',
 'de',
 'dedans',
 'dehors',
 'depuis',
 'des',
 'deux',
 'devoir',
 'devrait',
 'devrez',
 'devriez',
 'devrions',
 'devrons',
 'devront',
 'dois',
 'doit',
 'donc',
 'dos',
 'droite',
 'du',
 'dès',
 'début',
 'dù',
 'elle',
 'elles',
 'en',
 'encore',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'faire',
 'fais',
 'faisez',
 'fait',
 'faites',
 'fois',
 'font',
 'force',
 'furent',

In [9]:
def sent_to_words(sentences) :
    for sentence in sentences :
        yield(simple_preprocess(str(sentence), deacc = True))

In [10]:
doc_set = list_clean_content
data_words = list(sent_to_words(doc_set))

In [11]:
ndocs = len(data_words)
print(ndocs)

110895


In [12]:
stop_words = fr_stop_words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [13]:
# création du dictionnaire
dico = corpora.Dictionary(data_words_nostops)

# ce qui permet par ex. de filtrer le vocabulaire
dico.filter_extremes(no_below = 10)

# Create Corpus
texts = data_words_nostops

# matrice Term Document Frequency
corpus = [dico.doc2bow(text) for text in texts]

In [14]:
len(dico)

70949

In [15]:
ntopics = 50

In [16]:
generate_lda = True
#generate_lda = False

# generate LDA model
import logging

if generate_lda :
    print("generate new LDA model")
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)
    ldamodel = LdaModel( corpus, 
                         num_topics = ntopics, 
                         id2word = dico,
                         passes = 2, 
                         random_state = 100, 
                         per_word_topics = True)
    
    print(ldamodel)

2022-11-28 12:32:23,028 : INFO : using symmetric alpha at 0.02
2022-11-28 12:32:23,036 : INFO : using symmetric eta at 0.02
2022-11-28 12:32:23,062 : INFO : using serial LDA version on this node


generate new LDA model


2022-11-28 12:32:23,389 : INFO : running online (multi-pass) LDA training, 50 topics, 2 passes over the supplied corpus of 110895 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2022-11-28 12:32:23,389 : INFO : PROGRESS: pass 0, at document #2000/110895
2022-11-28 12:32:25,326 : INFO : merging changes from 2000 documents into a model of 110895 documents
2022-11-28 12:32:25,508 : INFO : topic #28 (0.020): 0.006*"plus" + 0.006*"temps" + 0.005*"etre" + 0.005*"quelques" + 0.004*"meme" + 0.004*"bien" + 0.004*"livre" + 0.003*"celui" + 0.003*"vie" + 0.003*"apres"
2022-11-28 12:32:25,508 : INFO : topic #46 (0.020): 0.006*"plus" + 0.005*"etre" + 0.005*"entre" + 0.004*"meme" + 0.004*"quelques" + 0.003*"monde" + 0.003*"vie" + 0.003*"parfois" + 0.003*"bien" + 0.003*"ete"
2022-11-28 12:32:25,508 : INFO : topic #12 (0.020): 0.010*"plus" + 0.007*"bien" + 0.007*"meme" + 0.006*"jean" + 0.004*"etre" 

2022-11-28 12:32:38,665 : INFO : topic #24 (0.020): 0.035*"the" + 0.025*"we" + 0.020*"in" + 0.019*"by" + 0.015*"re" + 0.012*"mc" + 0.010*"heart" + 0.009*"etc" + 0.009*"eme" + 0.009*"of"
2022-11-28 12:32:38,665 : INFO : topic #41 (0.020): 0.020*"songs" + 0.015*"ministre" + 0.013*"your" + 0.010*"still" + 0.010*"etc" + 0.009*"please" + 0.008*"ans" + 0.008*"help" + 0.007*"by" + 0.006*"patti"
2022-11-28 12:32:38,665 : INFO : topic #18 (0.020): 0.022*"www" + 0.012*"com" + 0.012*"fr" + 0.007*"etc" + 0.007*"plus" + 0.007*"parlement" + 0.006*"over" + 0.006*"poesie" + 0.005*"mots" + 0.004*"langue"
2022-11-28 12:32:38,665 : INFO : topic #45 (0.020): 0.020*"charline" + 0.015*"linda" + 0.015*"etc" + 0.014*"god" + 0.010*"euh" + 0.007*"joplin" + 0.007*"records" + 0.006*"janis" + 0.006*"parson" + 0.006*"book"
2022-11-28 12:32:38,681 : INFO : topic diff=0.568043, rho=0.377964
2022-11-28 12:32:38,681 : INFO : PROGRESS: pass 0, at document #16000/110895
2022-11-28 12:32:40,250 : INFO : merging changes fr

2022-11-28 12:32:52,421 : INFO : topic #10 (0.020): 0.017*"halles" + 0.009*"unpetit" + 0.009*"dalles" + 0.008*"plus" + 0.007*"veau" + 0.006*"marre" + 0.005*"where" + 0.005*"meme" + 0.005*"alice" + 0.005*"etre"
2022-11-28 12:32:52,437 : INFO : topic diff=0.518576, rho=0.277350
2022-11-28 12:32:52,437 : INFO : PROGRESS: pass 0, at document #28000/110895
2022-11-28 12:32:54,854 : INFO : merging changes from 2000 documents into a model of 110895 documents
2022-11-28 12:32:55,012 : INFO : topic #13 (0.020): 0.013*"spectacles" + 0.011*"festival" + 0.007*"plus" + 0.007*"troupe" + 0.007*"veston" + 0.006*"cour" + 0.006*"surla" + 0.005*"dansl" + 0.005*"ateliers" + 0.005*"rue"
2022-11-28 12:32:55,012 : INFO : topic #6 (0.020): 0.051*"etc" + 0.015*"maman" + 0.015*"film" + 0.013*"fils" + 0.010*"aime" + 0.009*"ah" + 0.009*"cinema" + 0.008*"ans" + 0.008*"famille" + 0.007*"arte"
2022-11-28 12:32:55,012 : INFO : topic #45 (0.020): 0.010*"ebauche" + 0.008*"euh" + 0.007*"touriste" + 0.007*"angelique" + 0

...

2022-11-28 12:41:02,299 : INFO : topic #29 (0.020): 0.118*"signez" + 0.029*"contributions" + 0.021*"mathieu" + 0.019*"yemen" + 0.018*"jade" + 0.015*"po" + 0.013*"laure" + 0.012*"jerusalem" + 0.012*"ana" + 0.010*"fete"
2022-11-28 12:41:02,299 : INFO : topic #19 (0.020): 0.071*"until" + 0.029*"million" + 0.026*"far" + 0.012*"bonjour" + 0.012*"beta" + 0.011*"alex" + 0.010*"start" + 0.010*"incident" + 0.009*"ri" + 0.009*"duc"
2022-11-28 12:41:02,302 : INFO : topic #49 (0.020): 0.025*"souviens" + 0.023*"monsieur" + 0.009*"save" + 0.009*"changes" + 0.009*"suite" + 0.007*"ter" + 0.007*"geant" + 0.007*"fourmis" + 0.007*"li" + 0.006*"alarme"
2022-11-28 12:41:02,302 : INFO : topic #35 (0.020): 0.020*"etre" + 0.017*"meme" + 0.017*"plus" + 0.012*"bien" + 0.010*"temps" + 0.008*"chose" + 0.007*"dire" + 0.007*"rien" + 0.007*"toujours" + 0.007*"vie"
2022-11-28 12:41:02,319 : INFO : topic diff=0.166866, rho=0.131936
2022-11-28 12:41:02,323 : INFO : PROGRESS: pass 1, at document #68000/110895
2022-11-28

2022-11-28 12:41:11,313 : INFO : topic #42 (0.020): 0.160*"michel" + 0.107*"euros" + 0.061*"eric" + 0.060*"volkovitch" + 0.057*"blogs" + 0.034*"pedro" + 0.027*"dr" + 0.023*"rubrique" + 0.018*"pa" + 0.017*"felix"
2022-11-28 12:41:11,313 : INFO : topic #18 (0.020): 0.071*"http" + 0.065*"www" + 0.062*"com" + 0.032*"article" + 0.029*"poesie" + 0.026*"fr" + 0.025*"https" + 0.019*"revue" + 0.017*"poemes" + 0.016*"auteurs"
2022-11-28 12:41:11,329 : INFO : topic diff=0.090392, rho=0.131936
2022-11-28 12:41:13,246 : INFO : -13.356 per-word bound, 10485.2 perplexity estimate based on a held-out corpus of 2000 documents with 181846 words
2022-11-28 12:41:13,246 : INFO : PROGRESS: pass 1, at document #80000/110895
2022-11-28 12:41:14,452 : INFO : merging changes from 2000 documents into a model of 110895 documents
2022-11-28 12:41:14,610 : INFO : topic #22 (0.020): 0.050*"del" + 0.046*"no" + 0.044*"el" + 0.039*"al" + 0.022*"los" + 0.021*"then" + 0.018*"mi" + 0.014*"las" + 0.013*"para" + 0.013*"ver

2022-11-28 12:41:21,068 : INFO : topic diff=0.099429, rho=0.131936
2022-11-28 12:41:21,083 : INFO : PROGRESS: pass 1, at document #92000/110895
2022-11-28 12:41:21,680 : INFO : merging changes from 2000 documents into a model of 110895 documents
2022-11-28 12:41:21,838 : INFO : topic #45 (0.020): 0.136*"thomas" + 0.028*"reine" + 0.026*"ue" + 0.025*"book" + 0.023*"brautigan" + 0.022*"tom" + 0.018*"givre" + 0.016*"investigation" + 0.015*"gabrielle" + 0.012*"sebastian"
2022-11-28 12:41:21,853 : INFO : topic #3 (0.020): 0.018*"homme" + 0.016*"mort" + 0.015*"vie" + 0.014*"hommes" + 0.011*"amour" + 0.010*"femme" + 0.010*"monde" + 0.010*"guerre" + 0.008*"femmes" + 0.008*"enfant"
2022-11-28 12:41:21,853 : INFO : topic #41 (0.020): 0.100*"police" + 0.056*"your" + 0.050*"help" + 0.037*"out" + 0.032*"just" + 0.029*"violence" + 0.022*"allies" + 0.021*"media" + 0.020*"justice" + 0.018*"here"
2022-11-28 12:41:21,853 : INFO : topic #31 (0.020): 0.088*"org" + 0.020*"wikipedia" + 0.016*"fr" + 0.014*"ar

2022-11-28 12:41:32,033 : INFO : merging changes from 2000 documents into a model of 110895 documents
2022-11-28 12:41:32,190 : INFO : topic #14 (0.020): 0.059*"new" + 0.046*"york" + 0.031*"david" + 0.028*"secrets" + 0.025*"roger" + 0.022*"james" + 0.020*"jose" + 0.019*"maurice" + 0.016*"di" + 0.015*"colette"
2022-11-28 12:41:32,190 : INFO : topic #2 (0.020): 0.136*"the" + 0.076*"and" + 0.074*"to" + 0.066*"of" + 0.026*"for" + 0.025*"that" + 0.025*"is" + 0.024*"in" + 0.020*"it" + 0.019*"you"
2022-11-28 12:41:32,190 : INFO : topic #4 (0.020): 0.042*"theatre" + 0.041*"scene" + 0.040*"musique" + 0.021*"spectacle" + 0.019*"voix" + 0.015*"piece" + 0.014*"chanson" + 0.012*"chansons" + 0.009*"danse" + 0.009*"joue"
2022-11-28 12:41:32,190 : INFO : topic #1 (0.020): 0.089*"plus" + 0.025*"monde" + 0.007*"toutes" + 0.007*"idees" + 0.006*"aujourd" + 0.006*"hui" + 0.005*"meme" + 0.005*"toute" + 0.005*"etre" + 0.004*"autres"
2022-11-28 12:41:32,190 : INFO : topic #37 (0.020): 0.030*"chat" + 0.017*"cl

LdaModel(num_terms=70949, num_topics=50, decay=0.5, chunksize=2000)


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare( ldamodel,
                                      gensim.matutils.corpus2csc(corpus),
                                      dictionary = ldamodel.id2word)

pyLDAvis.display(vis)

C:\Users\filip\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
